In [2]:
import tkinter as tk
from tkinter import messagebox

# Onsight Competition Score Application

## Goals
[ ] Get the old 25/15/10/5 scoring rules
[ ] Create the logic for the scoring system
[ ] Create the GUI for information input
[ ] Add a layer to the existing code for the scoring system
[ ] Set up error messages
[ ] Final testing
[ ] Push to features

In [1]:
import tkinter as tk
from tkinter import ttk, messagebox

# OOP Scoring System logic
class Scores:
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops} T    {self.attempts} A"
    
    class Zone:
        def __init__(self, zones = 0, attempts = 0):
            self.zones = zones
            self.attempts = attempts

        def __str__(self):
            return f"{self.zones} Z    {self.attempts} A"

    class LowZone:
        def __init__(self, low_zones = 0, attempts = 0):
            self.low_zones = low_zones
            self.attempts = attempts

        def __str__(self):
            return f"{self.low_zones} LZ    {self.attempts} A"

    def __init__(self):
        self.tops = Scores.Top(0, 0)
        self.zones = Scores.Zone(0, 0)
        self.low_zones = Scores.LowZone(0, 0)

class Climber:
    def __init__(self, name, category):
        self.name = name
        self.scores = Scores()
        self.category = category

    def __str__(self):
        return f"{self.name}:\n{self.scores.tops}\n{self.scores.zones}\n{self.scores.low_zones}"
    
    def delete(self, leaderboard):
        """Deletes the climber from the leaderboard"""
        leaderboard.climbers.remove(self)
    
class Leaderboard():
    def __init__(self):
        self.climbers = []

    def add_climber(self, *climbers):
        """Adds a climber to the leaderboard"""
        self.climbers.extend(climbers)
    
    def rank_climbers(self):
        """Sorts climbers by their scores. Key currently matches the USA Climbing Isolation scoring system."""
        def sort_key(climber):
            return (-climber.scores.tops.tops, -climber.scores.zones.zones, -climber.scores.low_zones.low_zones, 
            climber.scores.tops.attempts, climber.scores.zones.attempts, climber.scores.low_zones.attempts)
        self.climbers.sort(key=sort_key)
    
def create_climber(name, category, tops = (0, 0), zones = (0, 0), low_zones = (0, 0)):
    """Creates a climber object with the given name, category, and scores. Scores are taken as tuples.
        If no Score is provided, the name is added to the leaderboard with 0,0"""
    climber = Climber(name, category)
    climber.scores.tops = Scores.Top(*tops)
    climber.scores.zones = Scores.Zone(*zones)
    climber.scores.low_zones = Scores.LowZone(*low_zones)
    return climber

# tkinter window
class ScoringApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        self.leaderboard = Leaderboard()
        
        # Main Frame
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")
        self.main_frame.rowconfigure(0, weight= 1)
        self.main_frame.columnconfigure(0, weight= 1)
        self.main_frame.columnconfigure(1, weight= 1)

        # Resizing to fit the frame
        self.root.columnconfigure(0, weight= 1)
        self.root.rowconfigure(0, weight= 1)

        # Creating Left and Right sections
        self.create_left_frame()
        self.create_right_frame()

    def create_left_frame(self):
        """Create the left frame for inputs and buttons"""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, pady= 5, sticky= 'nw')

        # Left Frame Resizing
        #self.main_frame.columnconfigure(0, weight= 1)
        self.left_frame.columnconfigure(0, weight= 1)
        self.left_frame.columnconfigure(1, weight= 1)
        self.left_frame.rowconfigure(0, weight= 1)
        self.left_frame.rowconfigure(5, weight= 1)

        # Labels and Entry Fields
        labels = ["Climber Name", "Category", "Tops (count, attempts)", "Zones (count, attempts)", "Low Zones (count, attempts)"]
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.left_frame, text= label + ":").grid(row=i, column=0, padx=5, pady=5)
            entry = ttk.Entry(self.left_frame)
            entry.grid(row= i, column= 1, padx=5, pady=5, sticky= "w")
            self.entries[label] = entry #stores in dictionary for easy access
        
        # Leftside Buttons
        ttk.Button(self.left_frame, text="Add Climber", command=self.add_climber).grid(row= 5, column= 0, columnspan= 2, pady= 10)

        # Edit/Remove Climber Buttons
        add_remove_buttons = [
            ("Edit Climber", self.edit_climber),
            ("Remove Climber", self.remove_climber)
        ]
        for i, (text, command) in enumerate(add_remove_buttons):
            ttk.Button(self.left_frame, text= text, command= command).grid(row= 6, column= i, columnspan= 2, padx= 10, pady= 10, sticky= "w")

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and associated buttons"""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky= 'nsew')

        #letting right frame expand to fill the space
        self.main_frame.columnconfigure(1, weight= 1)
        self.right_frame.rowconfigure(0, weight= 1)
        self.right_frame.columnconfigure(0, weight= 1)

        # Leaderboard Display
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=30)
        self.leaderboard_text.grid(row=0, column=0, columnspan= 2, pady=10, sticky= "nsew")
        self.disable_edits()

        # Rightside Buttons
        self.leaderboard_buttons = ttk.Frame(self.right_frame)
        self.leaderboard_buttons.grid(row=1, column=0, columnspan=2, pady=10)
        ttk.Button(self.leaderboard_buttons, text="Clear Leaderboard", command=self.clear_leaderboard_ask).pack(side= "left", padx= 10, pady= 10)
        ttk.Button(self.leaderboard_buttons, text="Show Leaderboard", command=self.show_leaderboard).pack(side= "left", padx= 10, pady= 10)

    # Functions and Logic
    def clear_entries(self):
        """Clears all entry fields"""
        for entry in self.entries.values():
            entry.delete(0, tk.END)

    def remove_climber(self):
        """Removes a climber from the Leaderboard"""
        try:
            name = self.entries["Climber Name"].get()
            found = False
            if name == "":
                raise ValueError("Climber Name cannot be empty")
            for climber in self.leaderboard.climbers[:]:
                if climber.name == name:
                    climber.delete(self.leaderboard)
                    found = True
                    break
            if found:
                messagebox.showinfo("Success", f"{name} has been removed.")
                self.clear_entries()
                self.show_leaderboard()
            else:
                raise ValueError(f"Climber {name} not found in the leaderboard.")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def validate_scores(self, score_type):
        """Parses and validates the score from the entry field. Raises error an incorrect score is give"""
        entry_value = self.entries[score_type].get()
        scores = tuple(map(int, entry_value.split(',')))
        if not entry_value:
            return (0, 0)
        short_types = score_type.split()[0]
        if scores[0] > scores[1]:
            raise ValueError(f"{short_types}: '{scores[0]}' cannot be larger than attempts '{scores[1]}'")
        return scores

    def add_climber(self):
        """Adds a climber to the current leaderboard. Raises errors if the input is invalid"""
        try:
            name = self.entries["Climber Name"].get()
            if name == "":
                raise ValueError("Name cannot be empty")
            if any(climber.name == name for climber in self.leaderboard.climbers):
                raise ValueError(f"{name} already exists in the leaderboard")
            category = self.entries["Category"].get()
            tops = self.validate_scores("Tops (count, attempts)")
            zones = self.validate_scores("Zones (count, attempts)")
            low_zones = self.validate_scores("Low Zones (count, attempts)") if self.entries["Low Zones (count, attempts)"].get() else (0, 0)
   
            climber = create_climber(name, category, tops, zones, low_zones)
            self.leaderboard.add_climber(climber)
            messagebox.showinfo("Success", f"{name} added to the leaderboard!")
            self.clear_entries()
            self.show_leaderboard()
        except Exception as e:
            messagebox.showerror("Error", f"Invalid input: {e}")

    def show_leaderboard(self):
        """Displays the current leaderboard, with rankings"""
        self.leaderboard.rank_climbers()
        self.enable_edits()
        self.leaderboard_text.delete(1.0, tk.END)
        for climber in self.leaderboard.climbers:
            self.leaderboard_text.insert(tk.END, f"{climber}\n\n")
        self.disable_edits()

    def edit_climber(self):
        """Edits the score of an existing climber.
            If a new value is entered in the entry field, it is validated and updated"""
        try: 
            name = self.entries["Climber Name"].get()  
            if not name:
                raise ValueError("Climber Name cannot be empty")  
             
            climber = next((c for c in self.leaderboard.climbers if c.name == name), None)
            if not climber:
                raise ValueError(f"Climber {name} not found.")
            
            category = self.entries["Category"].get()
            if category:
                climber.category = category
            
            if self.entries["Tops (count, attempts)"].get():
                tops = self.validate_scores("Tops (count, attempts)")
                climber.scores.tops = Scores.Top(*tops)

            if self.entries["Zones (count, attempts)"].get():
                zones = self.validate_scores("Zones (count, attempts)")
                climber.scores.zones = Scores.Zone(*zones)
        
            if self.entries["Low Zones (count, attempts)"].get():
                low_zones = self.validate_scores("Low Zones (count, attempts)")
                climber.scores.low_zones = Scores.LowZone(*low_zones)

            messagebox.showinfo("Success", f"Climber '{name}' has been updated.")
            self.clear_entries()
            self.show_leaderboard()
        except Exception as e:
            messagebox.showerror("Error", f"Error updating climber '{name}':\n{str(e)}")

    def clear_leaderboard_ask(self):
        """Asks for confirmation before clearing the entries"""
        response = messagebox.askyesno("Clear Leaderboard", "Are you sure you want to clear the leaderboard?")
        if response:
            self.clear_leaderboard()
        
    def clear_leaderboard(self):
        """Clears the leaderboard values"""
        self.leaderboard.climbers = []
        self.enable_edits()
        self.leaderboard_text.delete(1.0, tk.END)
        messagebox.showinfo("Success", "Leaderboard has been cleared.")
        self.disable_edits()

    def enable_edits(self):
        self.leaderboard_text.config(state= tk.NORMAL)

    def disable_edits(self):
        self.leaderboard_text.config(state= tk.DISABLED)

if __name__ == "__main__":
    root = tk.Tk()
    app = ScoringApp(root)
    root.mainloop()

In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
# Creating the starting window to select the different scoring types.
# Creating (currently 2) scoring frames, when one is selected, use .tkraise() to bring selected scoring method to the forefront of the UI
# Next step is to flush out the UI for each scoring style
class App(tk.Tk):
    """Base level window that holds the other frames"""
    def __init__(self):
        super().__init__()
        self.title("Scoring App")

        container = tk.Frame(self)
        container.pack(fill= "both", expand= True)

        self.frames = {}

        for F in (StartingWindow, ScoringUSAC, ScoringOlympic):
            frame = F(container, self)
            self.frames[F.__name__] = frame
            frame.grid(row=0, column=0, sticky="nsew")

        self.show_frame("StartingWindow")

    def show_frame(self, name):
        self.frames[name].tkraise()

class StartingWindow(tk.Frame):
    """First window where the type of scoring is selected"""
    def __init__(self, parent, controller):
        super().__init__(parent)

        
        # Window
        self.starting_frame = ttk.Frame(self)
        self.starting_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= 'nsew')
        self.grid_columnconfigure(0, weight= 1)
        self.grid_rowconfigure(0, weight= 1)

        # Text
        self.choice_text = ttk.Label(self.starting_frame, text= "Please select your Scoring Style!")
        self.choice_text.grid(column= 0, row= 0, padx= 20, pady= 20, columnspan= 2)
        self.starting_frame.columnconfigure((0, 1), weight= 1)
        self.starting_frame.rowconfigure(1, weight= 1)

        # Buttons
        self.button_USAC = ttk.Button(self.starting_frame, text= "USAC Zone Style", command= lambda: controller.show_frame("ScoringUSAC"))
        self.button_USAC.grid(column= 0, row= 1, ipadx= 20, ipady= 20, sticky= 'ew')

        self.button_Olympic = ttk.Button(self.starting_frame, text= "Olympic Style", command= lambda: controller.show_frame("ScoringOlympic"))
        self.button_Olympic.grid(column= 1, row= 1, ipadx= 20, ipady= 20, sticky= 'ew')


class ScoringUSAC(tk.Frame):
    """The classic scoring window layout for USAC and IFSC events"""
    def __init__(self, parent, controller):
        super().__init__(parent)


        # Main frame
        self.main_frame = ttk.Frame(self)
        self.main_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")
        self.main_frame.rowconfigure(0, weight= 1)
        self.main_frame.columnconfigure(0, weight= 1)
        self.main_frame.columnconfigure(1, weight= 1)

        self.create_left()
        self.create_right()

        #ttk.Label(self.main_frame, text= "Welcome to USAC Scoring").grid(column= 0, row= 0)

        backbutton = ttk.Button(self.main_frame, text= "Back", command= lambda: controller.show_frame("StartingWindow"))
        backbutton.grid(column= 0, row= 1)

    def create_left(self):
        """Left side creation where labels and entry fields exist to gather information from the user"""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, pady= 5, sticky= 'nw')


        self.left_frame.columnconfigure(0, weight= 1)
        self.left_frame.columnconfigure(1, weight= 1)
        self.left_frame.rowconfigure(0, weight= 1)
        self.left_frame.rowconfigure(5, weight= 1)

        # Labels and Entry Fields
        labels = ["Climber Name", "Category", "Tops (count, attempts)", "Zones (count, attempts)", "Low Zones (count, attempts)"]
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.left_frame, text= label + ":").grid(row=i, column=0, padx=5, pady=5)
            entry = ttk.Entry(self.left_frame)
            entry.grid(row= i, column= 1, padx=5, pady=5, sticky= "w")
            self.entries[label] = entry #stores in dictionary for easy access
        
        # Leftside Buttons
        ttk.Button(self.left_frame, text="Add Climber").grid(row= 5, column= 0, columnspan= 2, pady= 10)

    def create_right(self):
        """Create the right frame for the leaderboard display and associated buttons"""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky= 'nsew')

        #letting right frame expand to fill the space
        self.main_frame.columnconfigure(1, weight= 1)
        self.right_frame.rowconfigure(0, weight= 1)
        self.right_frame.columnconfigure(0, weight= 1)

        # Leaderboard Display
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=30)
        self.leaderboard_text.grid(row=0, column=0, columnspan= 2, pady=10, sticky= "nsew")
        #self.disable_edits()

        # Rightside Buttons
        self.leaderboard_buttons = ttk.Frame(self.right_frame)
        self.leaderboard_buttons.grid(row=1, column=0, columnspan=2, pady=10)
        ttk.Button(self.leaderboard_buttons, text="Clear Leaderboard").pack(side= "left", padx= 10, pady= 10) # command=self.clear_leaderboard_ask
        ttk.Button(self.leaderboard_buttons, text="Show Leaderboard").pack(side= "left", padx= 10, pady= 10)

    # USAC Scoring logic
    class Scores:
        class Top:
            def __init__(self, tops = 0, attempts = 0):
                self.tops = tops
                self.attempts = attempts

            def __str__(self):
                return f"{self.tops} T    {self.attempts} A"
        
        class Zone:
            def __init__(self, zones = 0, attempts = 0):
                self.zones = zones
                self.attempts = attempts

            def __str__(self):
                return f"{self.zones} Z    {self.attempts} A"

        class LowZone:
            def __init__(self, low_zones = 0, attempts = 0):
                self.low_zones = low_zones
                self.attempts = attempts

            def __str__(self):
                return f"{self.low_zones} LZ    {self.attempts} A"

        def __init__(self):
            self.tops = Scores.Top(0, 0)
            self.zones = Scores.Zone(0, 0)
            self.low_zones = Scores.LowZone(0, 0)


class ScoringOlympic(tk.Frame):
    """Olympic Style scoring that is more points focused rather than zone focused"""
    def __init__(self, parent, controller):
        super().__init__(parent)
        ttk.Label(self, text= "you opened the olympic scoring").pack()

        backbutton = ttk.Button(self, text= "Back", command= lambda: controller.show_frame("StartingWindow"))
        backbutton.pack()


if __name__ == "__main__":
    app = App()
    #app.geometry("600x200")
    app.mainloop()

In [4]:
# Using Polymorphism with the scoring types.
class ScoringStyle:
    def __init__(self):
        """Initialize the scoring style."""
        pass

    def rank_climbers(self, climber):
        """Sorts climbers based on the scoring logic, which is implemented by the subclass."""
        raise NotImplementedError("Subclass must implement this method.")
    
class USAC_ScoringStyle(ScoringStyle):
    class Scores:
        class Top:
            def __init__(self, tops = 0, attempts = 0):
                self.tops = tops
                self.attempts = attempts

            def __str__(self):
                return f"{self.tops} T    {self.attempts} A"
        
        class Zone:
            def __init__(self, zones = 0, attempts = 0):
                self.zones = zones
                self.attempts = attempts

            def __str__(self):
                return f"{self.zones} Z    {self.attempts} A"

        class LowZone:
            def __init__(self, low_zones = 0, attempts = 0):
                self.low_zones = low_zones
                self.attempts = attempts

            def __str__(self):
                return f"{self.low_zones} LZ    {self.attempts} A"

        def __init__(self):
            self.tops = USAC_ScoringStyle.Scores.Top(0, 0)
            self.zones = USAC_ScoringStyle.Scores.Zone(0, 0)
            self.low_zones = USAC_ScoringStyle.Scores.LowZone(0, 0)

    def rank_climbers(self, climbers):
        """Sorts climbers by their scores. Key currently matches the USA Climbing Isolation scoring system."""
        def sort_key(climber):
            return (-climber.scores.tops.tops, -climber.scores.zones.zones, -climber.scores.low_zones.low_zones, 
            climber.scores.tops.attempts, climber.scores.zones.attempts, climber.scores.low_zones.attempts)
        self.climbers.sort(key=sort_key)

class OlympicScores(ScoringStyle):
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops} T    {self.attempts} A"
        
        def calculate_score(self):
            return (25 * self.tops) - (max(0, self.attempts - self.tops) * 0.1)
    
    class Zone10:
        def __init__(self, zone10 = 0, attempts = 0):
            self.zone10 = zone10
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone10} Z10    {self.attempts} A"

        def calculate_score(self):
            return (10 * self.zone10) - (max(0, self.attempts - self.zone10) * 0.1)        

    class Zone5:
        def __init__(self, zone5 = 0, attempts = 0):
            self.zone5 = zone5
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone5} Z5    {self.attempts} A"

        def calculate_score(self):
            return (5 * self.zone5) - (max(0, self.attempts - self.zone5) * 0.1)

    class Total:
        def __init__(self, top = None, zone10 = None, zone5 = None):
            self.top = top if top else OlympicScores.Top()
            self.zone10 = zone10 if zone10 else OlympicScores.Zone10()
            self.zone5 = zone5 if zone5 else OlympicScores.Zone5()

            self.total_score = (
                self.top.calculate_score() +
                self.zone10.calculate_score() +
                self.zone5.calculate_score()
            )

        def __str__(self):
            return f"Total Score: {self.total_score: .1f}\n Tops Score: {self.top.calculate_score()}\n Zone10 Score: {self.zone10.calculate_score()}\n Zone5 Score: {self.zone5.calculate_score()}"


In [2]:
# Alternative Scoring System
    # Scoring using the 25/15/10/5 system found in older USAC competitions and some IFSC.
scoring_mode = "Olympic"
'''
Here's how the scores work:
If a climber tops a boulder, they are awarded 25 - ((attempts - 1) * 0.1) points. If top is not achieved, their added attempts/score is awarded.
when a climber reaches each scored space (15/10/5) they are awarded those points, minus .1 for each attempt taken. Only the highest scored value is taken.
With 4 boulders, there is a maximum of 100 points total. If there aren't enough zones for each number, then the higher number is taken.
Ranking is based on total score
'''
# Breakdown of how the scoring should look, Using max here to make sure tops isn't larger than attempts
"""
top_score = (25 * num_of_tops) - (max(0, attempts_to_top - num_of_tops) * 0.1)
zone10_score = (10 * num_of_zone10) - (max(0, attempts_to_zone10 - num_of_zone10) * 0.1)
zone5_score = (5 * num_of_zone5) - (max(0, attempts_to_zone5 - num_of_zone5) * 0.1)
Total_score = top_score + zone10_score + zone5_score
"""


class OlympicScores:
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops} T    {self.attempts} A"
        
        def calculate_score(self):
            return (25 * self.tops) - (max(0, self.attempts - self.tops) * 0.1)
    
    class Zone10:
        def __init__(self, zone10 = 0, attempts = 0):
            self.zone10 = zone10
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone10} Z10    {self.attempts} A"

        def calculate_score(self):
            return (10 * self.zone10) - (max(0, self.attempts - self.zone10) * 0.1)        

    class Zone5:
        def __init__(self, zone5 = 0, attempts = 0):
            self.zone5 = zone5
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone5} Z5    {self.attempts} A"

        def calculate_score(self):
            return (5 * self.zone5) - (max(0, self.attempts - self.zone5) * 0.1)

    class Total:
        def __init__(self, top = None, zone10 = None, zone5 = None):
            self.top = top if top else OlympicScores.Top()
            self.zone10 = zone10 if zone10 else OlympicScores.Zone10()
            self.zone5 = zone5 if zone5 else OlympicScores.Zone5()

            self.total_score = (
                self.top.calculate_score() +
                self.zone10.calculate_score() +
                self.zone5.calculate_score()
            )

        def __str__(self):
            return f"Total Score: {self.total_score: .1f}\n Tops Score: {self.top.calculate_score()}\n Zone10 Score: {self.zone10.calculate_score()}\n Zone5 Score: {self.zone5.calculate_score()}"

# Written as (scored zones/tops, attempts to zones/tops)
top = OlympicScores.Top(2, 6)
zone10 = OlympicScores.Zone10(3, 8)
zone5 = OlympicScores.Zone5(4, 9)

total = OlympicScores.Total(top, zone10, zone5)
print(total)
  

class Climber:
    def __init__(self, name):
        self.name = name
        self.scores = OlympicScores()

    def __str__(self):
        return f"{self.name}: {self.scores.Total} points."


#print(f"Total Score: {total_score}")

Total Score:  98.6
 Tops Score: 49.6
 Zone10 Score: 29.5
 Zone5 Score: 19.5


In [3]:
""" NEEDS WORK """
#Teasing out scoring errors. A score should only be earned by the highest point a climber got to in each boulder
#  and then -.1 point for each unsucessful attempt

class OlympicScores:
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops} T    {self.attempts} A"
        
        def calculate_score(self):
            return (25 * self.tops) - (max(0, self.attempts - self.tops) * 0.1)
    
    class Zone10:
        def __init__(self, zone10 = 0, attempts = 0):
            self.zone10 = zone10
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone10} Z10    {self.attempts} A"

        def calculate_score(self):
            return (10 * self.zone10) - (max(0, self.attempts - self.zone10) * 0.1)        

    class Zone5:
        def __init__(self, zone5 = 0, attempts = 0):
            self.zone5 = zone5
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone5} Z5    {self.attempts} A"

        def calculate_score(self):
            return (5 * self.zone5) - (max(0, self.attempts - self.zone5) * 0.1)

    class Total:
        def __init__(self, top = None, zone10 = None, zone5 = None):
            self.top = top if top else OlympicScores.Top()
            self.zone10 = zone10 if zone10 else OlympicScores.Zone10()
            self.zone5 = zone5 if zone5 else OlympicScores.Zone5()

            self.total_score = (
                self.top.calculate_score() +
                self.zone10.calculate_score() +
                self.zone5.calculate_score()
            )

        def __str__(self):
            return f"Total Score: {self.total_score: .1f}\n Tops Score: {self.top.calculate_score()}\n Zone10 Score: {self.zone10.calculate_score()}\n Zone5 Score: {self.zone5.calculate_score()}"

# Written as (scored zones/tops, attempts to zones/tops)
top = OlympicScores.Top(2, 6)
zone10 = OlympicScores.Zone10(3, 8)
zone5 = OlympicScores.Zone5(4, 9)

total = OlympicScores.Total(top, zone10, zone5)
print(total)
  

class Climber:
    def __init__(self, name):
        self.name = name
        self.scores = OlympicScores()

    def __str__(self):
        return f"{self.name}: {self.scores.Total} points."


#print(f"Total Score: {total_score}")

Total Score:  98.6
 Tops Score: 49.6
 Zone10 Score: 29.5
 Zone5 Score: 19.5


In [ ]:
# Working with a dropdown menu to select which type of scoring system to use
import tkinter as tk
from tkinter import ttk, messagebox

options = ["Olympic Scoring", "Zone Based Scoring", "5/10/15/25 Scoring"]
class TestWindow:
    def __init__(self, root):
        self.root = root
        self.root.title("Testing Default Window")
        self.main_page = ttk.Frame(self.root)
        self.main_page.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")
        self.root.geometry("400x400")

        self.selected_value = tk.StringVar(root)
        self.selected_value.set("Choose a scoring type")
        self.dropdown_menu = ttk.OptionMenu(self.main_page, self.selected_value, self.selected_value.get(), *options)
        self.dropdown_menu.grid(column= 0, row= 1, padx= 20, pady= 20)

        self.submit_button = ttk.Button(self.main_page, text= "Submit", command= self.update_frame)
        self.submit_button.grid(column= 0, row= 2)

        self.dynamic_frame = ttk.Frame(self.main_page)
        self.dynamic_frame.grid(column= 0, row= 3, pady= 10)

    def update_frame(self):
        """Updates the dynamic frame based on the selected scoring option."""
        for widget in self.dynamic_frame.winfo_children():
            widget.destroy()

        # Get the selected option
        selected_option = self.selected_value.get()

        if selected_option == "Olympic Scoring":
            OlympicScoring(self.dynamic_frame)
        elif selected_option == "Zone Based Scoring":
            ZoneBasedScoring(self.dynamic_frame)
        else:
            ttk.Label(self.dynamic_frame, text= "Please select a valid option").grid(column=0, row=0, pady=10)

class OlympicScoring:
    """GUI Window for entering information about Olympic Style Scoring"""
    def __init__(self, parent):
        self.parent = parent

        # GUI Input fields for Olympic Scoring
        ttk.Label(parent, text= "Name: ").grid(column= 0, row= 0)
        self.name = ttk.Entry(parent, width= 10).grid(column= 1, row= 0, columnspan= 2)

        ttk.Label(parent, text="Tops (count, attempts):").grid(column=0, row=1, sticky="w")
        self.tops_count = ttk.Entry(parent, width=10)
        self.tops_count.grid(column=1, row=1, padx=5)
        self.tops_attempts = ttk.Entry(parent, width=10)
        self.tops_attempts.grid(column=2, row=1, padx=5)

        ttk.Label(parent, text="Zone10 (count, attempts):").grid(column=0, row=2, sticky="w")
        self.zone10_count = ttk.Entry(parent, width=10)
        self.zone10_count.grid(column=1, row=2, padx=5)
        self.zone10_attempts = ttk.Entry(parent, width=10)
        self.zone10_attempts.grid(column=2, row=2, padx=5)

        ttk.Label(parent, text="Zone5 (count, attempts):").grid(column=0, row=3, sticky="w")
        self.zone5_count = ttk.Entry(parent, width=10)
        self.zone5_count.grid(column=1, row=3, padx=5)
        self.zone5_attempts = ttk.Entry(parent, width=10)
        self.zone5_attempts.grid(column=2, row=3, padx=5)

        # Submit button to calculate the score
        self.calculate_button = ttk.Button(parent, text="Calculate Score", command=self.calculate_score)
        self.calculate_button.grid(column=0, row=4, pady=10, columnspan=3)

        # Label to display the result
        self.result_label = ttk.Label(parent, text="")
        self.result_label.grid(column=0, row=5, pady=10, columnspan=3)

    def calculate_score(self):
        tops = int(self.tops_count.get())
        tops_attempts = int(self.tops_attempts.get())
        zone10 = int(self.zone10_count.get())
        zone10_attempts = int(self.zone10_attempts.get())
        zone5 = int(self.zone5_count.get())
        zone5_attempts = int(self.zone5_attempts.get())

        # storing data for final scores
        top_score = OlympicScores.Top(tops, tops_attempts)
        zone10_score = OlympicScores.Zone10(zone10, zone10_attempts)
        zone5_score = OlympicScores.Zone5(zone5, zone5_attempts)
        total_score = OlympicScores.Total(top_score, zone10_score, zone5_score)

        # Display results
        self.result_label.config(
            text= f"Total Score: {total_score.total_score:.1f}\nScore From Tops: {top_score}\nScore from Zone10: {zone10_score}"
                                  )

class OlympicScores:
    """Logic for Olympic Scores"""
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops} T    {self.attempts} Attempts"
        
        def calculate_score(self):
            return (25 * self.tops) - (max(0, self.attempts - self.tops) * 0.1)
    
    class Zone10:
        def __init__(self, zone10 = 0, attempts = 0):
            self.zone10 = zone10
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone10} Z10    {self.attempts} Attempts"

        def calculate_score(self):
            return (10 * self.zone10) - (max(0, self.attempts - self.zone10) * 0.1)        

    class Zone5:
        def __init__(self, zone5 = 0, attempts = 0):
            self.zone5 = zone5
            self.attempts = attempts

        def __str__(self):
            return f"{self.zone5} Z5    {self.attempts} Attempts"

        def calculate_score(self):
            return (5 * self.zone5) - (max(0, self.attempts - self.zone5) * 0.1)

    class Total:
        def __init__(self, top = None, zone10 = None, zone5 = None):
            self.top = top if top else OlympicScores.Top()
            self.zone10 = zone10 if zone10 else OlympicScores.Zone10()
            self.zone5 = zone5 if zone5 else OlympicScores.Zone5()

            self.total_score = (
                self.top.calculate_score() +
                self.zone10.calculate_score() +
                self.zone5.calculate_score()
            )

        def __str__(self):
            return f"Total Score: {self.total_score: .1f}\n Tops Score: {self.top.calculate_score()}\n Zone10 Score: {self.zone10.calculate_score()}\n Zone5 Score: {self.zone5.calculate_score()}"


class ZoneBasedScoring:
    def __init__(self, parent):
        ttk.Label(parent, text= "Zone Based Scoring Selected").grid(column= 0, row= 0, pady= 10)

if __name__ == "__main__":
    root = tk.Tk()
    app = TestWindow(root)
    root.mainloop()

In [11]:
# Testing Cell
import tkinter as tk
from tkinter import ttk, messagebox

# OOP Scoring System logic
class Scores:
    class Top:
        def __init__(self, tops = 0, attempts = 0):
            self.tops = tops
            self.attempts = attempts

        def __str__(self):
            return f"{self.tops} T    {self.attempts} A"
    
    class Zone:
        def __init__(self, zones = 0, attempts = 0):
            self.zones = zones
            self.attempts = attempts

        def __str__(self):
            return f"{self.zones} Z    {self.attempts} A"

    class LowZone:
        def __init__(self, low_zones = 0, attempts = 0):
            self.low_zones = low_zones
            self.attempts = attempts

        def __str__(self):
            return f"{self.low_zones} LZ    {self.attempts} A"

    def __init__(self):
        self.tops = Scores.Top(0, 0)
        self.zones = Scores.Zone(0, 0)
        self.low_zones = Scores.LowZone(0, 0)

class Climber:
    def __init__(self, name, category):
        self.name = name
        self.scores = Scores()
        self.category = category

    def __str__(self):
        return f"{self.name}:\n{self.scores.tops}\n{self.scores.zones}\n{self.scores.low_zones}"
    
    def delete(self):
        """Deletes the climber from the leaderboard"""
        del self
    
class Leaderboard():
    def __init__(self):
        self.climbers = []

    def add_climber(self, *climbers):
        """Adds a climber to the leaderboard"""
        self.climbers.extend(climbers)
    
    def rank_climbers(self):
        """Sorts climbers by their scores. Key currently matches the USA Climbing Isolation scoring system."""
        def sort_key(climber):
            return (-climber.scores.tops.tops, -climber.scores.zones.zones, -climber.scores.low_zones.low_zones, 
            climber.scores.tops.attempts, climber.scores.zones.attempts, climber.scores.low_zones.attempts)
        self.climbers.sort(key=sort_key)
    
def create_climber(name, category, tops, zones, low_zones = (0, 0)):
    """Creates a climber object with the given name, category, and scores. Low zones are optional and scores are taken as tuples."""
    climber = Climber(name, category)
    climber.scores.tops = Scores.Top(*tops)
    climber.scores.zones = Scores.Zone(*zones)
    climber.scores.low_zones = Scores.LowZone(*low_zones)
    return climber

# tkinter window
class ScoringApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        self.leaderboard = Leaderboard()
        
        # Main Frame
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")
        self.main_frame.rowconfigure(0, weight= 1)
        self.main_frame.columnconfigure(0, weight= 1)
        self.main_frame.columnconfigure(1, weight= 1)

        # Resizing to fit the frame
        self.root.columnconfigure(0, weight= 1)
        self.root.rowconfigure(0, weight= 1)

        # Creating Left and Right sections
        self.create_left_frame()
        self.create_right_frame()

    def create_left_frame(self):
        """Create the left frame for inputs and buttons"""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, sticky= 'nw')

        # Left Frame Resizing
        self.left_frame.columnconfigure(0, weight= 1)
        self.left_frame.columnconfigure(1, weight= 1)
        self.left_frame.rowconfigure(0, weight= 1)
        self.left_frame.rowconfigure(5, weight= 1)

        # Labels and Entry Fields
        labels = ["Climber Name", "Category", "Tops (count, attempts)", "Zones (count, attempts)", "Low Zones (count, attempts)"]
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.left_frame, text= label + ":").grid(row=i, column=0, padx=5, pady=5)
            entry = ttk.Entry(self.left_frame)
            entry.grid(row= i, column= 1, padx=5, pady=5, sticky= "w")
            self.entries[label] = entry #stores in dictionary for easy access
        
        # Leftside Buttons
        ttk.Button(self.left_frame, text="Add Climber", command=self.add_climber).grid(row= 5, column= 0, columnspan= 2, pady= 10)

        # Edit/Remove Climber Buttons
        add_remove_buttons = [
            ("Edit Climber", self.edit_climber),
            ("Remove Climber", self.remove_climber)
        ]
        for i, (text, command) in enumerate(add_remove_buttons):
            ttk.Button(self.left_frame, text= text, command= command).grid(row= 6, column= i, columnspan= 2, padx= 10, pady= 10, sticky= "w")

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and associated buttons"""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky= 'nsew')

        #letting right frame expand to fill the space
        self.main_frame.columnconfigure(1, weight= 1)
        self.right_frame.rowconfigure(0, weight= 1)
        self.right_frame.columnconfigure(0, weight= 1)

        # Leaderboard Display
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=30)
        self.leaderboard_text.grid(row=0, column=0, columnspan= 2, pady=10, sticky= "nsew")
        self.disable_edits()

        # Rightside Buttons
        self.leaderboard_buttons = ttk.Frame(self.right_frame)
        self.leaderboard_buttons.grid(row=1, column=0, columnspan=2, pady=10)
        ttk.Button(self.leaderboard_buttons, text="Clear Leaderboard", command=self.clear_leaderboard_ask).pack(side= "left", padx= 10, pady= 10)
        ttk.Button(self.leaderboard_buttons, text="Show Leaderboard", command=self.show_leaderboard).pack(side= "left", padx= 10, pady= 10)

    # Functions and Logic
    def clear_entries(self):
        """Clears all entry fields"""
        self.enable_edits()
        for entry in self.entries.values():
            entry.delete(0, tk.END)
        self.disable_edits()

    def remove_climber(self):
        """Removes a climber from the Leaderboard"""
        try:
            name = self.entries["Climber Name"].get()
            found = False
            if name == "":
                raise ValueError("Climber Name cannot be empty")
            for climber in self.leaderboard.climbers[:]:
                if climber.name == name:
                    self.leaderboard.climbers.remove(climber)
                    climber.delete()
                    found = True
                    break
            if found:
                messagebox.showinfo("Success", f"{name} has been removed.")
                self.clear_entries()
                self.show_leaderboard()
            else:
                raise ValueError(f"Climber {name} not found in the leaderboard.")
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def validate_scores(self, score_type):
        """Parses and validates the score from the entry field. Raises error an incorrect score is give"""
        scores = tuple(map(int, self.entries[score_type].get().split(',')))
        short_types = score_type.split()[0]
        if scores[0] > scores[1]:
            raise ValueError(f"{short_types}: '{scores[0]}' cannot be larger than attempts '{scores[1]}'")
        return scores

    def add_climber(self):
        """Adds a climber to the current leaderboard. Raises errors if the input is invalid"""
        try:
            name = self.entries["Climber Name"].get()
            if name == "":
                raise ValueError("Name cannot be empty")
            if name in [climber.name for climber in self.leaderboard.climbers]:
                raise ValueError(f"{name} already exists in the leaderboard")
            category = self.entries["Category"].get()
            tops = self.validate_scores("Tops (count, attempts)")
            zones = self.validate_scores("Zones (count, attempts)")
            low_zones = self.validate_scores("Low Zones (count, attempts)") if self.entries["Low Zones (count, attempts)"].get() else (0, 0)
   
            climber = create_climber(name, category, tops, zones, low_zones)
            self.leaderboard.add_climber(climber)
            messagebox.showinfo("Success", f"{name} added to the leaderboard!")
            self.clear_entries()
            self.show_leaderboard()
        except Exception as e:
            messagebox.showerror("Error", f"Invalid input: {e}")

    def show_leaderboard(self):
        """Displays the current leaderboard, with rankings"""
        self.leaderboard.rank_climbers()
        self.enable_edits()
        self.leaderboard_text.delete(1.0, tk.END)
        for climber in self.leaderboard.climbers:
            self.leaderboard_text.insert(tk.END, f"{climber}\n\n")
        self.disable_edits()

    def edit_climber(self):
        """Edits the score of an existing climber.
            If a new value is entered in the entry field, it is validated and updated"""
        try: 
            name = self.entries["Climber Name"].get()  
            if not name:
                raise ValueError("Climber Name cannot be empty")  
             
            climber = next((c for c in self.leaderboard.climbers if c.name == name), None)
            if not climber:
                raise ValueError(f"Climber {name} not found.")
            
            category = self.entries["Category"].get()
            if category:
                climber.category = category
            
            if self.entries["Tops (count, attempts)"].get():
                tops = self.validate_scores("Tops (count, attempts)")
                climber.scores.tops = Scores.Top(*tops)

            if self.entries["Zones (count, attempts)"].get():
                zones = self.validate_scores("Zones (count, attempts)")
                climber.scores.zones = Scores.Zone(*zones)
        
            if self.entries["Low Zones (count, attempts)"].get():
                low_zones = self.validate_scores("Low Zones (count, attempts)")
                climber.scores.low_zones = Scores.LowZone(*low_zones)

            messagebox.showinfo("Success", f"Climber '{name}' has been updated.")
            self.clear_entries()
            self.show_leaderboard()
        except Exception as e:
            messagebox.showerror("Error", f"Error updating climber '{name}':\n{str(e)}")

    def clear_leaderboard_ask(self):
        """Asks for confirmation before clearing the entries"""
        response = messagebox.askyesno("Clear Leaderboard", "Are you sure you want to clear the leaderboard?")
        if response:
            self.clear_leaderboard()
        
    def clear_leaderboard(self):
        """Clears the leaderboard values"""
        self.leaderboard.climbers = []
        self.enable_edits()
        self.leaderboard_text.delete(1.0, tk.END)
        messagebox.showinfo("Success", "Leaderboard has been cleared.")
        self.disable_edits()

    def enable_edits(self):
        self.leaderboard_text.config(state= tk.NORMAL)

    def disable_edits(self):
        self.leaderboard_text.config(state= tk.DISABLED)

if __name__ == "__main__":
    root = tk.Tk()
    app = ScoringApp(root)
    root.mainloop()

In [ ]:
# old layout that still works but isn't OOP
# tkinter window
import tkinter as tk
from tkinter import ttk, messagebox

class ScoringApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        self.leaderboard = Leaderboard()
        
        # Main Frame
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.grid(column= 0, row= 0, padx= 20, pady= 20, sticky= "nsew")

        # Resizing to fit the frame
        self.root.columnconfigure(0, weight= 1)
        self.root.columnconfigure(1, weight= 1)
        self.root.rowconfigure(0, weight= 1)

        # Creating Left and Right sections
        self.create_left_frame()
        self.create_right_frame()

    def create_left_frame(self):
        """Create the left frame for inputs and buttons"""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, sticky= 'nw')

        # Labels and Entry Fields
        labels = ["Climber Name", "Category", "Tops (count, attempts)", "Zones (count, attempts)", "Low Zones (count, attempts)"]
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.left_frame, text= label + ":").grid(row=i, column=0, padx=5, pady=5)
            entry = ttk.Entry(self.left_frame)
            entry.grid(row= i, column= 1, padx=5, pady=5, sticky= "w")
            self.entries[label] = entry #stores in dictionary for easy access

            # Climber Name
        ttk.Label(self.left_frame, text="Climber Name:").grid(row=0, column=0, padx=5, pady=5)
        self.name_entry = ttk.Entry(self.left_frame)
        self.name_entry.grid(row=0, column=1, padx=5, pady=5, sticky= "w")

            # Climber Category
        ttk.Label(self.left_frame, text="Category:").grid(row=1, column=0, padx=5, pady=5)
        self.category_entry = ttk.Entry(self.left_frame)
        self.category_entry.grid(row=1, column= 1, padx=5, pady=5, sticky= "w")
        
            # Climber Scores, Tops, Zones, Low Zones
        ttk.Label(self.left_frame, text="Tops (count, attempts):").grid(row=2, column=0, padx=5, pady=5)
        self.tops_entry = ttk.Entry(self.left_frame)
        self.tops_entry.grid(row=2, column= 1, padx=5, pady=5, sticky= "w")
        
        ttk.Label(self.left_frame, text="Zones (count, attempts):").grid(row=3, column=0, padx=5, pady=5)
        self.zones_entry = ttk.Entry(self.left_frame)
        self.zones_entry.grid(row=3, column=1, padx=5, pady=5, sticky= "w")
        
        ttk.Label(self.left_frame, text="Low Zones (count, attempts):").grid(row=4, column=0, padx=5, pady=5)
        self.low_zones_entry = ttk.Entry(self.left_frame)
        self.low_zones_entry.grid(row=4, column=1, padx=5, pady=5, sticky= "w")

        
        # Leftside Buttons
        ttk.Button(self.left_frame, text="Add Climber", command=self.add_climber).grid(row= 5, column= 0, columnspan= 2, pady= 10)
        ttk.Button(self.left_frame, text="Edit Climber").grid(row= 6, column= 0, columnspan= 2, padx= 10, pady= 10, sticky= "w")
        ttk.Button(self.left_frame, text= "Remove Climber").grid(row= 6, column= 1, columnspan= 2, padx= 10, pady= 10, sticky= "e")

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and associated buttons"""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky= 'ne')

        # Leaderboard Display
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=50)
        self.leaderboard_text.grid(row=0, column=0, columnspan= 2, pady=10, sticky= "nsew")

        # Rightside Buttons
        ttk.Button(self.right_frame, text="Clear Leaderboard", command=self.clear_leaderboard_ask).grid(row= 1, column= 0, columnspan= 2, pady= 10)
        ttk.Button(self.right_frame, text="Show Leaderboard", command=self.show_leaderboard).grid(row= 1, column= 1, columnspan= 2, pady= 10)

    # Functions and Logic
    def clear_entries(self):
        """Clears all but category entry fields"""
        self.name_entry.delete(0, tk.END)
        self.tops_entry.delete(0, tk.END)
        self.zones_entry.delete(0, tk.END)
        self.low_zones_entry.delete(0, tk.END)

    def add_climber(self):
        """Adds a climber to the current leaderboard"""
        try:
            name = self.name_entry.get()
            category = self.category_entry.get()
            tops = tuple(map(int, self.entries["Tops (count, attempts)"].get().split(',')))
            zones = tuple(map(int, self.entries["Zones (count, attempts)"].get().split(',')))
            low_zones = tuple(map(int, self.entries["Low Zones (count, attempts)"].get().split(',')) if self.entries["Low Zones (count, attempts)"].get() else (0, 0))

            
            climber = create_climber(name, category, tops, zones, low_zones)
            self.leaderboard.add_climber(climber)
            messagebox.showinfo("Success", f"{name} added to the leaderboard!")
            self.clear_entries()
        except Exception as e:
            messagebox.showerror("Error", f"Invalid input: {e}")

    def show_leaderboard(self):
        """Displays the current leaderboard, with rankings"""
        self.leaderboard.rank_climbers()
        self.leaderboard_text.delete(1.0, tk.END)
        for climber in self.leaderboard.climbers:
            self.leaderboard_text.insert(tk.END, f"{climber}\n\n")

    def edit_climber(self):
        """Edits the score of an existing climber"""
        try: 
            name = self.name_entry.get()
            category = self.category_entry.get()
            tops = tuple(map(int, self.tops_entry.get().split(',')))
            zones = tuple(map(int, self.zones_entry.get().split(',')))
            low_zones = tuple(map(int, self.low_zones_entry.get().split(','))) if self.low_zones_entry.get() else (0, 0)

            for climber in self.leaderboard.climbers:
                if climber.name == name:
                    climber.scores.tops = Scores.Top(*tops)
                    climber.scores.zones = Scores.Zone(*zones)
                    climber.scores.low_zones = Scores.LowZone(*low_zones)
                    messagebox.showinfo("Success", f"{name} has been updated.")
                    self.clear_entries()
                    return
        except Exception as e:
            messagebox.showerror("Error", f"invalid input {e}")

    def clear_leaderboard_ask(self):
        """Asks for confirmation before clearing the entries"""
        response = messagebox.askyesno("Clear Leaderboard", "Are you sure you want to clear the leaderboard?")
        if response:
            self.clear_leaderboard()
        
    def clear_leaderboard(self):
        """Clears the leaderboard values"""
        self.leaderboard.climbers = []
        self.leaderboard_text.delete(1.0, tk.END)
        messagebox.showinfo("Success", "Leaderboard has been cleared.")

if __name__ == "__main__":
    root = tk.Tk()
    app = ScoringApp(root)
    root.mainloop()


In [45]:
# Leaderboard to the right
class LeaderboardApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        
        # Create Main Frame to Hold Everything
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.pack(expand=True, fill="both", padx=20, pady=20)

        # Create Left and Right Sections
        self.create_left_frame()
        self.create_right_frame()

    def create_left_frame(self):
        """Create the left frame for inputs and buttons."""
        self.left_frame = ttk.Frame(self.main_frame)
        self.left_frame.grid(row=0, column=0, padx=10, sticky="nw")

        labels = ["Climber Name:", "Tops (count, attempts):", 
                  "Zones (count, attempts):", "Low Zones (count, attempts):"]
        
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.left_frame, text=label).grid(row=i, column=0, padx=5, pady=5, sticky="e")
            entry = ttk.Entry(self.left_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky="w")
            self.entries[label] = entry  # Store in dictionary for easy access

        # Button to Add Climber (Stays on Left Side)
        ttk.Button(self.left_frame, text="Add Climber").grid(row=len(labels), column=0, columnspan=2, pady=10)

        # Bottom Buttons (Only "Edit" and "Remove" stay on left)
        self.bottom_frame = ttk.Frame(self.left_frame)
        self.bottom_frame.grid(row=len(labels)+1, column=0, columnspan=2, pady=10)

        ttk.Button(self.bottom_frame, text="Edit Climber").grid(row=0, column=0, padx=5)
        ttk.Button(self.bottom_frame, text="Remove Climber").grid(row=0, column=1, padx=5)

    def create_right_frame(self):
        """Create the right frame for the leaderboard display and leaderboard-related buttons."""
        self.right_frame = ttk.Frame(self.main_frame)
        self.right_frame.grid(row=0, column=1, padx=10, sticky="n")

        # Leaderboard Text Box
        self.leaderboard_text = tk.Text(self.right_frame, height=20, width=50)
        self.leaderboard_text.pack(expand=True, fill="both")

        # Buttons Below the Leaderboard
        self.leaderboard_buttons_frame = ttk.Frame(self.right_frame)
        self.leaderboard_buttons_frame.pack(pady=10)

        ttk.Button(self.leaderboard_buttons_frame, text="Show Leaderboard").pack(side="left", padx=5)
        ttk.Button(self.leaderboard_buttons_frame, text="Clear Leaderboard").pack(side="left", padx=5)

if __name__ == "__main__":
    root = tk.Tk()
    app = LeaderboardApp(root)
    root.mainloop()

In [42]:
# Leaderboard below 
class LeaderboardApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Climbing Leaderboard")
        
        # Configure root layout for expansion
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(2, weight=1)  # Make the leaderboard expand

        # Create Frames
        self.create_input_frame()
        self.create_button_frame()
        self.create_leaderboard()
        self.create_bottom_buttons()

    def create_input_frame(self):
        """Create the input fields for climber details."""
        self.input_frame = ttk.Frame(self.root)
        self.input_frame.grid(row=0, column=0, padx=10, pady=10, sticky="ew")

        labels = ["Climber Name:", "Category:", "Tops (count, attempts):", 
                  "Zones (count, attempts):", "Low Zones (count, attempts):"]
        
        self.entries = {}
        for i, label in enumerate(labels):
            ttk.Label(self.input_frame, text=label).grid(row=i, column=0, padx=5, pady=5, sticky="e")
            entry = ttk.Entry(self.input_frame, width=30)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky="w")
            self.entries[label] = entry  # Store in dictionary for easy access

    def create_button_frame(self):
        """Create buttons for adding and showing leaderboard."""
        self.button_frame = ttk.Frame(self.root)
        self.button_frame.grid(row=1, column=0, pady=10)

        ttk.Button(self.button_frame, text="Add Climber").grid(row=0, column=0, padx=5)
        ttk.Button(self.button_frame, text="Show Leaderboard").grid(row=0, column=1, padx=5)

    def create_leaderboard(self):
        """Create the text box for displaying the leaderboard."""
        self.leaderboard_text = tk.Text(self.root, height=15, width=50)
        self.leaderboard_text.grid(row=2, column=0, pady=10, padx=10, sticky="nsew")

    def create_bottom_buttons(self):
        """Create edit and clear buttons at the bottom."""
        self.bottom_frame = ttk.Frame(self.root)
        self.bottom_frame.grid(row=3, column=0, pady=10)

        ttk.Button(self.bottom_frame, text="Edit Climber").grid(row=0, column=0, padx=5)
        ttk.Button(self.bottom_frame, text="Clear Leaderboard").grid(row=0, column=1, padx=5)
        ttk.Button(self.bottom_frame, text="Remove Climber").grid(row=0, column=2, padx=5)

if __name__ == "__main__":
    root = tk.Tk()
    app = LeaderboardApp(root)
    root.mainloop()